<a href="https://colab.research.google.com/github/sulaimanbehzad/Classifying-Images/blob/main/Image_Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifiers
The purpose of this project is to train two classifiers:
1. Captions classifier
2. Image classifier

In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [69]:
# function for reading text files into a pandas dataframe
def read_data_into_df(filespath):
  levels = 0
  for roots, dirnames, filenames in os.walk(filespath):
    if levels == 0:
      df = pd.DataFrame(columns=dirnames)
    txt_list = []
    for fn in filenames:
      txtfile_full_path = os.path.join(roots, fn)
      print(txtfile_full_path)
      with open(txtfile_full_path) as f:
        txt_list.append(f.read())
    root = os.path.split(roots) 
    root = root[1]
    print(root)
    df[root] = txt_list
    levels+=1
  return df


In [70]:
path_train_sentences = r'/content/drive/MyDrive/dataset/train/sentences'   
path_test_sentences =  r'/content/drive/MyDrive/dataset/test/sentences'   
train = read_data_into_df(path_train_sentences)
test = read_data_into_df(path_test_sentences)

sentences
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004501.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004506.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004301.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004097.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004550.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_005094.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004930.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004910.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004908.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_005008.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_004004.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_003609.txt
/content/drive/MyDrive/dataset/train/sentences/tvmonitor/2008_003995.txt
/content/drive/MyDrive/dataset/train/sent

In [71]:
train.head()

,tvmonitor,train,sofa,sheep,pottedplant,person,motorbike,horse,dog,diningtable,cow,chair,cat,car,bus,boat,bird,bicycle,aeroplane,sentences
0,A man sitting at an office desk with his compu...,An old passenger train in blue and white sits ...,There are two people sitting on a purple couch...,A boy feeding a sheep hay.\nA small boy feeds ...,A black and white cat along with a purple Afri...,A guy in a sports jersey poses with a girl in ...,A helmet wearing motorcycle stuntman performs ...,A horse rider is riding a white horse.\na man ...,The caged dogs are spread from the others.\nTw...,A kitchendiner with papers laying on the blue ...,Cattle grazing on a snowy field.\nCattle in a ...,A group of red plastic chairs in a hall.\nRed ...,A cat is looking out the window.\nA cat lookin...,A hand painted Volkswagon Bug.\nA multicolored...,A yellow taxi passing two school buses.\nThere...,A boat with four sails is sailing on water.\nA...,An American Eagle standing on a post with its ...,A bicycle is parked on a trail among trees.\nA...,An Air Canada airplane flying in the sky.\nAn ...,NaN
1,A home office with three computer monitors and...,A black steam train traveling on the railroad ...,An empty college common space with empty chair...,A black sheep and a white sheep stand on grass...,A small house with a broken door and windows.\...,A baby is sitting in the grass.\nA baby with b...,A grey haired man inspects a red Suzuki motorc...,A brown on in the field on a sunny day.\nA clo...,A black dog in a grass field.\nA black dog sta...,A close-up of a head table set with gray napki...,A black and white cow grazing near water.\nA c...,A kitchen table in a room with white drapes an...,A car sits by a window with a curtain pulled m...,A bus stop in front of a red brick building.\n...,An elderly woman catches a ride on the back of...,A car with a canoe on top is parked on the str...,A blue and green hummingbird hovers near a pin...,A little girl with a bike helmet posing by her...,A small plane grounded in a field of grass.\nA...,NaN
2,A boy looking at a computer screen.\nA boy loo...,A passenger train coming into a station.\nA su...,A blue couch with sparse surroundings.\nA blue...,An old man with sheep in the background.\nan o...,"A unique living room with tv, brown table, gre...",A person jumping a dirt bike with a boy watchi...,A close-up of a woman with black boots next to...,A jockey rides a brown and white horse in a di...,A brown and white dog laying on the bed while ...,A close up of the center of a dining room tabl...,"A brown and white horse with its eyes closed, ...","A cubicle with a large, black office chair and...",A gray cat laying on a brown table.\nA grey ca...,A nice silver Toyota faces me on a brick drive...,A park garden in front of red trolleys and a l...,A man stands while paddling a small canoe.\nA ...,A beautiful striped bird on a barren limb.\nA ...,A boy doing a wheelie on a plank with the beac...,An Air Canada airplane is ascending against a ...,NaN
3,A cluttered living space is typical of a small...,A blue and yellow train crosses a river near a...,An overweight woman in pajamas and bunny slipp...,A domestic animal stands in a pen.\nA sheep st...,a girl holding a black white dog with a blue d...,A dark haired woman with green eyes and a blue...,A close up of the front end of a purple and wh...,A jockey races on the number 6 gray horse.\na ...,"A small white dog standing, wearing some dog-c...",A kitchendining room with a computer workstati...,A child on a pink stool milking a black and wh...,An smiling asian child sits beside a young wom...,A brown cat sits by a window.\nA graceful cat ...,A bicyclist explaining arm pain to a fellow ri...,A car and three buses in a rear-view mirror.\n...,A docked fishing boat on a clear day.\na fishi...,A large bird is flying through the air.\nA lar...,Two bicyclists and a pedestrian are waiting to...,A blue and yellow plan flies against a blue sk...,NaN
4,A bathtub under stained gla

In [72]:
test.head()

,tvmonitor,train,sofa,sheep,pottedplant,person,motorbike,horse,dog,diningtable,cow,chair,cat,car,bus,boat,bird,bicycle,aeroplane,sentences
0,A man in a Ninja Turtles t-shirt repairs telev...,A man posing in front of a passenger train.\nA...,A living room with wooden floors and patterned...,A group of sheep and a sheep dog in a large fi...,"Three dark haired young women in white, red an...",A small boy with curly black hair sits at a ta...,A black motorbike parked in front of a city wa...,A pitched tent with a horse in the background....,A man walking a dog on the beach near large wa...,A dinner table set for three people.\nA Thanks...,A black and white cow in a pen with hay.\na bl...,A clear plastic chair.\nA clear plastic chair ...,A kitten looking at the camera.\nA kitten lyin...,a side view of a red double decker bus on a bu...,a yellow school bus in front of a house\nPeopl...,A bus parked by a harbor.\nA white bus with th...,Two birds fly by a fencepost.\nTwo birds flyin...,A man is alone on the back of a tandem bicycle...,A plane maneuvers at low altitude over the riv...,NaN
1,A close-up of a computer sitting on a desk wit...,A yellow and white train and one orange train ...,Two smiling young ladies sitting on a couch.\n...,A sheep with a tree in the foreground.\nSheep ...,Art items on a table.\nA women painting a chai...,A main standing under a road bridge.\nA person...,A motocross rider about to get back on his stu...,A close-up of a brown horse's head.\nA close-u...,A brown dog laying on a carpeted floor next to...,A man holding a baby and a woman eating at a t...,A cow walking through a field.\nA dark colored...,A black and white cat is sitting on a brown ch...,A small cat laying on a wooden beam looking up...,A large wooded expanse with a city in the back...,A school bus driving down a remote dirt road.\...,A man and two women are posing for the camera ...,birds sitting on abar\nClose up of two birds p...,A cyclist riding along a waterside path\nA per...,An airplane is flying over a tree in the blue ...,NaN
2,A cozy wood-paneled living room with a wood bu...,An American flag and another flag fly above a ...,The two dogs blend in with the stuff animals.\...,Distant view of two lambs perched on top of a ...,Herbs grow in terra cotta plants on a balcony....,A baby sits in a car seat looking upward\nA ba...,A motorcycle rider rides splashes mud.\nA pers...,"A black and white photo of three horses, their...",A man sitting on a bench looking at a dog on a...,A group of people sit around a table with food...,A brown cow on a farm.\nA cow with a tag in it...,A carpeted kitchen with the dining room in the...,A black and white cat laying on the ground in ...,Four people are rowing a boat near the shore.\...,a brown double decker tour bus on a street wit...,An ocean liner at sea.\nA picture of a large s...,A man and a boy take a serene walk on the beac...,A bicycle is scaling the stone wall.\nA BMX bi...,Airplane on runway in front of buildings.\nAn ...,NaN
3,A computer desk displays elvis memorabilia.\nA...,A coal train traveling down the tracks.\nA loc...,A black and wooden seating area.\nA black couc...,A black faced sheep\nA black headed sheep cove...,A happy man in a blue shirt is standing in fro...,A man is watching TV on the foot of his bed.\n...,Motorcycles withjackets\nParked motorcycles re...,A close up of a horse wearing a blue halter.\n...,A dog is lying at the bottom of a staircase ne...,A black table with white fuzzy chair.\nA dinin...,A cow drinking out of a water trough\nBrown an...,A chandelier over a dining room table.\nA dini...,A gray and white cat sits on a table.\nA wide-...,A two story building on a street with a white ...,A crowded double-decker bus station in the cit...,Couple sailing in a small sailboat.\nThe two s...,one white bird and a black and white bird on a...,Bikers lean into a turn during a race.\nRacing...,A large passenger plane on a landing strip.\nA...,NaN
4,A darkened desk with a desktop 

In [73]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tvmonitor    28 non-null     object 
 1   train        28 non-null     object 
 2   sofa         28 non-null     object 
 3   sheep        28 non-null     object 
 4   pottedplant  28 non-null     object 
 5   person       28 non-null     object 
 6   motorbike    28 non-null     object 
 7   horse        28 non-null     object 
 8   dog          28 non-null     object 
 9   diningtable  28 non-null     object 
 10  cow          28 non-null     object 
 11  chair        28 non-null     object 
 12  cat          28 non-null     object 
 13  car          28 non-null     object 
 14  bus          28 non-null     object 
 15  boat         28 non-null     object 
 16  bird         28 non-null     object 
 17  bicycle      28 non-null     object 
 18  aeroplane    28 non-null     object 
 19  sentences 

In [74]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tvmonitor    20 non-null     object 
 1   train        20 non-null     object 
 2   sofa         20 non-null     object 
 3   sheep        20 non-null     object 
 4   pottedplant  20 non-null     object 
 5   person       20 non-null     object 
 6   motorbike    20 non-null     object 
 7   horse        20 non-null     object 
 8   dog          20 non-null     object 
 9   diningtable  20 non-null     object 
 10  cow          20 non-null     object 
 11  chair        20 non-null     object 
 12  cat          20 non-null     object 
 13  car          20 non-null     object 
 14  bus          20 non-null     object 
 15  boat         20 non-null     object 
 16  bird         20 non-null     object 
 17  bicycle      20 non-null     object 
 18  aeroplane    20 non-null     object 
 19  sentences 

In [78]:
for i in train['chair']:
  print(i)

A group of red plastic chairs in a hall.
Red chair in a tiled room.
Red plastic chairs on a linoleum floor.
There are red chairs, a yellow wall, a bluegreen designed wall and a table in this room.
This brightly lit floorspace is littered with red plastic chairs.

A kitchen table in a room with white drapes and a china cabinet.
A table under the lamp is next to a curtained window.
Dining room containing a table and chairs along with a wooden cabinet.
Dining table and chairs sitting under chandelier.
The dining room table is ready for a meal.

A cubicle with a large, black office chair and a telephone on the desk.
An empty office chair in a cubicle.
An empty office cubicle.
An empty office cubicle with chair, desk and phone.
There is a telephone and a chair in the cubicle.

An smiling asian child sits beside a young women near a blue mosaic table.
Asian mother and daughter seated at a dinner table.
A small girl and a woman seated at a restaurant smile for the camera.
A women and small ch

In [ ]:
for index, row in train.iterrows():
  print(index, row)

In [80]:
df1 = train.transpose(copy=False)
df1.head(30)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
tvmonitor,A man sitting at an office desk with his compu...,A home office with three computer monitors and...,A boy looking at a computer screen.\nA boy loo...,A cluttered living space is typical of a small...,A bathtub under stained glass windows.\nA beau...,A man laying on a bed with beer cans on a near...,A close-up of a television monitor with a man ...,"A living room with TV, books, vacuum, and acce...",A computer with two monitors on a white desk.\...,A home office set up complete with a can of so...,A\nA large screen tv with news headlines.\nA m...,girls and one guy playingtwister\nThree people...,College students finally get the answer they'v...,A man sitting in a chair with a laptop on his ...,A messy bedroom.\nA pile of bunched-up sheets ...,A computer monitor has a picture of a cat in a...,"A black, flat screen television placed on the ...","A computer and collection of gadgets, includin...",A group of men sitting in a classroom.\nStuden...,A green lamp sits beside a television on a dre...,A silver television on a silver stand with vis...,A green painted office with a buster eaten pos...,A television and entertainment center.\nA wide...,A laptop and a PC at a workstation.\nA laptop ...,A man pressing a button on a television.\nA ma...,A woman and her dog watch the cameraman in the...,A close-up picture of a desk with a computer a...,a desk with a computer in a room\nA messy home...
train,An old passenger train in blue and white sits ...,A black steam train traveling on the railroad ...,A passenger train coming into a station.\nA su...,A blue and yellow train crosses a river near a...,A train car at a large group of railroad track...,A long American freight train near a station.\...,A green steam engine traveling on the railroad...,A London Underground train in a siding next to...,A man gets off the Peruvian train for a photo ...,A man waits on the platform as the light rail ...,A trail car on the tracks at a station.\nA tra...,A blue and yellow locomotive alongside a stati...,A small train stopped on the tracks in a park....,An indoor train depot with trains on both side...,A CN freight engine passes by a larger freight...,A modern train in the snow.\nA train going dow...,A black steam engine with yellow stars on it.\...,A long train is on train tracks.\nA yellow fre...,A BNSF railroad train on a track by water.\nA ...,A black and white photo of an empty train stat...,A boxcar with graffiti on it.\nA graffiti cove...,A colorful train\nAn old American steam engine...,A close up of a silver train.\nA close-up of a...,An Amtrak passenger train waiting at a station...,Green train arriving at station.\nPassangers w...,"A red train\nA red, yellow and black locomotiv...",A multi-colored train on a track.\nan old dies...,A large yellow locomotive showing the number 8...
sofa,There are two people sitting on a purple couch...,An empty college common space with empty chair...,A blue couch with sparse surroundings.\nA blue...,An overweight woman in pajamas and bunny slipp...,A man and small child in a living room posing ...,"A blue, red, and white couch against a wall wi...","A small efficiency apartment, complete with tw...",A black leather couch with a chew' lounge and ...,A dining table in front of a window with two s...,A baby is being played with.\nA close-up of a ...,Teenagers hanging out in their room.\nTwo guys...,A girl on a couch looking to the woman sitting...,A couple posing for a picture on a couch.\nA m...,a dark living room with a white couch and a fi...,A white colored room with closed blinds and tw...,A bright red couch in a room with mostly woode...,A black and white photo of a living room with ...,A cramped and cluttered living room contains a...,Four Asian young people sitting in a den or li...,A women holds a baby on a sofa with a small bo...,A woman is lying across the laps of two men si...,a living room with a birdcage in the left 